In [1]:
!nvidia-smi

Tue Jun  4 12:03:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:3B:00.0 Off |                    0 |
| N/A   46C    P0             163W / 165W |  13444MiB / 24576MiB |     82%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(os.environ["CUDA_VISIBLE_DEVICES"])

1


In [3]:
from pathlib import Path
import ast
import json
import numpy as np
import pandas as pd

# change eval_on to use eval_lang column info 

In [102]:
def desired_cols(multi_idx):
    first_part = [i[0] for i in multi_idx]
    second_part = [i[1].split('-') for i in multi_idx]
    idx = []
    counter = range(len(second_part))
    for n,i,j in zip(counter,first_part, second_part):
        # print('',i,j,n)
        if (i=='metric') or ('all' in i) or ('all' in j):
            # print('-',i,j,n)
            pass
        else:
            if (i==j[1]) or (j[0]==j[1]):
                # print('--',i,j,n)
                idx.append(n)
    return idx

In [103]:
def data_cleanup (foldername, root, run=1, lc=False, n_split=3):
    DATA_PATH = root.joinpath(foldername)
    print(DATA_PATH)
    df = pd.read_json(DATA_PATH.joinpath('evaluations.jsonl'), lines=True)
    df.head()
    
    # clean up model name to determine trained on and tested on languages
    df['eval_on'] = df['model_name'].str.split('eval').apply(lambda x: x[-1])
    df['model_name'] = df['model_name'].str.split('_')
    df['trained_on'] = df['model_name'].apply(lambda x: x[1])
    df['model_name'] = df['model_name'].apply(lambda x: x[0])
    
    # isolate each item of classification report
    df['macro_avg_precision'] = [i['macro avg']['precision'] for i in df['class_report']]
    df['macro_avg_recall'] = [i['macro avg']['recall'] for i in df['class_report']]
    df['support'] = [i['macro avg']['support'] for i in df['class_report']]
    df['f1_1. Likely ILI infection'] = [i['1. Likely ILI infection']['f1-score'] for i in df['class_report']]
    df['f1_3. Not Related to ILI or COVID-19 Infection'] = [i['3. Not Related to ILI or COVID-19 Infection']['f1-score'] for i in df['class_report']]
    df['support_1. Likely ILI infection'] = [i['1. Likely ILI infection']['support'] for i in df['class_report']]
    df['support_3. Not Related to ILI or COVID-19 Infection'] = [i['3. Not Related to ILI or COVID-19 Infection']['support'] for i in df['class_report']]
    df.head()

    if lc:
        splits = [i for i in range(1,n_split+1)]
        # print(splits)
        start = 1
        skip = df[['trained_on', 'eval_on']].value_counts().sum()/n_split
        skip = skip.astype(np.int64)
        n_exp = []
        for i in splits:
            # print(i)
            end = skip*i
            # print(start,end)
            n_exp.extend([i for _ in range(start,end+1)])
            start = end+1
        if len(n_exp)==len(df):
            df['splits'] = n_exp
        else:
            print(f'length of dataframe {len(df)} and split list is {len(n_exp)} not equal')
        # df['splits'] = [1 for _ in range(1,n+1)] + [2 for _ in range(n+1,(n+1)*2)] + [3 for _ in range((n+1)*2,(n+1)*3)]
        # df = df[df['eval_on'].str.contains('all|es-es', regex=True, case=False)]

    df['run'] = run
    return df

In [109]:
ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2')
# ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/evals_mono/eval_revisedcateg/testset0.6_0.2_0.2')
# ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/final_evals_copy')

# exp = 'original' + '_predictions'
exp = 'one_to_many' + '_predictions'
print(exp)

n_runs = list(range(1,6))
n_runs

one_to_many_predictions


[1, 2, 3, 4, 5]

## For Regular Runs

In [110]:
all_runs = []
for i in n_runs:
    all_runs.append(data_cleanup(f'{exp}_run{i}', root=ROOT, run=i))
all_results = pd.concat(all_runs)
all_results['run'].unique()

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_predictions_run1
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_predictions_run2
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_predictions_run3
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_predictions_run4
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_predictions_run5


array([1, 2, 3, 4, 5])

In [111]:
f1_macro = pd.pivot_table(all_results, values='f1_macro', columns=['trained_on','eval_on'], index='model_name',  aggfunc = 'mean')
recall = pd.pivot_table(all_results, values='macro_avg_recall', columns=['trained_on','eval_on'], index='model_name',  aggfunc = 'mean')
precision = pd.pivot_table(all_results, values='macro_avg_precision', columns=['trained_on','eval_on'], index='model_name',  aggfunc = 'mean')
accuracy = pd.pivot_table(all_results, values='accuracy', columns=['trained_on','eval_on'], index='model_name',  aggfunc = 'mean')
support = pd.pivot_table(all_results, values='support', columns=['trained_on','eval_on'], index='model_name',  aggfunc = 'mean')
f1_macro['metric'] = 'f1_macro'
recall['metric'] = 'recall'
precision['metric'] = 'precision'
accuracy['metric'] = 'accuracy'
support['metric'] = 'support'
all_eval = pd.concat([f1_macro, recall, precision, accuracy,support])
all_eval.head()

trained_on                            de                                \
eval_on                              all     de-de     de-en     de-es   
model_name                                                               
bernice                         0.768612  0.816931  0.740265  0.741308   
bert-base-multilingual-uncased  0.687575  0.768862  0.721340  0.735906   
mdeberta-v3-base                0.757516  0.818759  0.722869  0.727702   
twitter-xlm-roberta-base        0.744984  0.809623  0.724910  0.745744   
xlm-roberta-base                0.752017  0.814206  0.747105  0.749139   

trained_on                                                              \
eval_on                            de-fr     de-it     en-de     en-en   
model_name                                                               
bernice                         0.737952  0.725910  0.685628  0.743926   
bert-base-multilingual-uncased  0.716765  0.729621  0.622502  0.596346   
mdeberta-v3-base                0.720471  0.720310  0.610611  0.664369   
twitter-xlm-roberta-base        0.755621  0.741114  0.648307  0.646370   
xlm-roberta-base                0.737556  0.739908  0.655993  0.648179   

trained_on                                          ...        it            \
eval_on                            en-es     en-fr  ...     fr-en     fr-es   
model_name                                          ...                       
bernice                         0.762341  0.715413  ...  0.761047  0.773646   
bert-base-multilingual-uncased  0.647116  0.604918  ...  0.724814  0.718473   
mdeberta-v3-base                0.671729  0.648259  ...  0.752579  0.739268   
twitter-xlm-roberta-base        0.672129  0.645665  ...  0.785298  0.787578   
xlm-roberta-base                0.630213  0.645349  ...  0.751241  0.740613   

trained_on                                                              \
eval_on                            fr-fr     fr-it     it-de     it-en   
model_name                                                               
bernice                         0.756136  0.730652  0.788356  0.811124   
bert-base-multilingual-uncased  0.724947  0.656016  0.794066  0.777750   
mdeberta-v3-base                0.747899  0.717921  0.757159  0.743713   
twitter-xlm-roberta-base        0.771625  0.760367  0.819951  0.863462   
xlm-roberta-base                0.726227  0.732295  0.776693  0.812182   

trained_on                                                      metric  
eval_on                            it-es     it-fr     it-it            
model_name                                                              
bernice                         0.813187  0.783241  0.798270  f1_macro  
bert-base-multilingual-uncased  0.814746  0.794470  0.762611  f1_macro  
mdeberta-v3-base                0.784219  0.750627  0.749656  f1_macro  
twitter-xlm-roberta-base        0.868614  0.852459  0.854009  f1_macro  
xlm-roberta-base                0.824621  0.800940  0.804504  f1_macro  

[5 rows x 131 columns]

In [34]:
all_eval.to_csv(ROOT.joinpath(f'{exp}_avg_of_5runs.csv'))

In [112]:
avg_metrics = all_eval.groupby('metric').mean()
selected_idx = desired_cols(avg_metrics.columns)
avg_metrics = avg_metrics[avg_metrics.columns[selected_idx]]
avg_metrics

trained_on          de                                                \
eval_on          de-de      en-de      en-en       es-de       es-es   
metric                                                                 
accuracy      0.806186   0.658000   0.664000    0.720858    0.755536   
f1_macro      0.805676   0.644608   0.659838    0.709749    0.750301   
precision     0.809249   0.660871   0.669024    0.740264    0.764686   
recall        0.806231   0.649495   0.666061    0.716569    0.753313   
support     194.000000  40.000000  40.000000  233.000000  233.000000   

trained_on                                                          en  ...  \
eval_on          fr-de       fr-fr       it-de       it-it       de-de  ...   
metric                                                                  ...   
accuracy      0.708187    0.716269    0.866000    0.870400    0.762680  ...   
f1_macro      0.680158    0.689074    0.713535    0.726371    0.761673  ...   
precision     0.728303    0.735754    0.751337    0.753352    0.767044  ...   
recall        0.684590    0.694318    0.697725    0.720078    0.763001  ...   
support     193.000000  193.000000  200.000000  200.000000  194.000000  ...   

trained_on          fr          it                                    \
eval_on          it-it       de-de       de-it      en-en      en-it   
metric                                                                 
accuracy      0.885600    0.790309    0.710722   0.705000   0.699000   
f1_macro      0.780008    0.787945    0.702767   0.701974   0.678678   
precision     0.778590    0.799054    0.727931   0.707334   0.714598   
recall        0.784471    0.788812    0.707940   0.704949   0.682929   
support     200.000000  194.000000  194.000000  40.000000  40.000000   

trained_on                                                              
eval_on          es-es       es-it       fr-fr       fr-it       it-it  
metric                                                                  
accuracy      0.844635    0.809099    0.752332    0.740311    0.907800  
f1_macro      0.843545    0.804476    0.745367    0.719450    0.793810  
precision     0.848774    0.826822    0.752220    0.760207    0.848292  
recall        0.843591    0.805674    0.744329    0.719207    0.764039  
support     233.000000  233.000000  193.000000  193.000000  200.000000  

[5 rows x 45 columns]

In [113]:
avg_metrics.to_csv(ROOT.joinpath(f'avg_{exp}.csv'))

## For Learning Curve Runs

In [6]:
ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2')
# ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/evals_mono/eval_revisedcateg/testset0.6_0.2_0.2')
# ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/final_evals_copy')

exp = 'es_learningcurve' + '_predictions'
print(exp)

n_runs = list(range(1,6))
n_runs

es_learningcurve_predictions


[1, 2, 3, 4, 5]

In [7]:
# for learning curve runs
lc=True
all_runs = []
for i in n_runs:
    all_runs.append(data_cleanup(f'{exp}_run{i}', root=ROOT, run=i, lc=lc, n_split=3))
all_results = pd.concat(all_runs)
all_results['run'].unique()

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_run1
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_run2
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_run3
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_run4
/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_run5


array([1, 2, 3, 4, 5])

In [9]:
f1_macro = pd.pivot_table(all_results, values='f1_macro', columns=['trained_on','eval_on'], index=['splits','model_name'], aggfunc = 'mean')
recall = pd.pivot_table(all_results, values='macro_avg_recall', columns=['trained_on','eval_on'], index=['splits','model_name'], aggfunc = 'mean')
precision = pd.pivot_table(all_results, values='macro_avg_precision', columns=['trained_on','eval_on'], index=['splits','model_name'], aggfunc = 'mean')
accuracy = pd.pivot_table(all_results, values='accuracy', columns=['trained_on','eval_on'], index=['splits','model_name'], aggfunc = 'mean')
support = pd.pivot_table(all_results, values='support', columns=['trained_on','eval_on'], index=['splits','model_name'], aggfunc = 'mean')
f1_macro['metric'] = 'f1_macro'
recall['metric'] = 'recall'
precision['metric'] = 'precision'
accuracy['metric'] = 'accuracy'
support['metric'] = 'support'
all_eval = pd.concat([f1_macro, recall, precision, accuracy,support])
all_eval.head()

trained_on                                   es                                \
eval_on                                     all     de-de     de-en     de-es   
splits model_name                                                               
1      bernice                         0.518841  0.443645  0.444375  0.600558   
       bert-base-multilingual-uncased  0.426793  0.487376  0.430427  0.429027   
       mdeberta-v3-base                0.283930  0.328720  0.328720  0.328720   
       twitter-xlm-roberta-base        0.804349  0.778923  0.733858  0.727907   
       xlm-roberta-base                0.305866  0.343629  0.333175  0.346724   

trained_on                                                                     \
eval_on                                   de-fr     de-it     en-de     en-en   
splits model_name                                                               
1      bernice                         0.608932  0.538297  0.382882  0.356614   
       bert-base-multilingual-uncased  0.464888  0.439815  0.405742  0.377625   
       mdeberta-v3-base                0.328720  0.328720  0.310345  0.310345   
       twitter-xlm-roberta-base        0.726933  0.732556  0.767898  0.750814   
       xlm-roberta-base                0.342464  0.334948  0.316408  0.323867   

trained_on                                                 ...            \
eval_on                                   en-es     en-fr  ...     fr-en   
splits model_name                                          ...             
1      bernice                         0.494362  0.560320  ...  0.397100   
       bert-base-multilingual-uncased  0.408529  0.409570  ...  0.424264   
       mdeberta-v3-base                0.310345  0.310345  ...  0.305755   
       twitter-xlm-roberta-base        0.759467  0.709873  ...  0.791114   
       xlm-roberta-base                0.345974  0.327397  ...  0.330938   

trained_on                                                                     \
eval_on                                   fr-es     fr-fr     fr-it     it-de   
splits model_name                                                               
1      bernice                         0.525261  0.552734  0.543226  0.329608   
       bert-base-multilingual-uncased  0.430967  0.460316  0.451445  0.291303   
       mdeberta-v3-base                0.305755  0.305755  0.305755  0.130435   
       twitter-xlm-roberta-base        0.806441  0.767220  0.777997  0.746962   
       xlm-roberta-base                0.330190  0.342427  0.326449  0.162291   

trained_on                                                                     \
eval_on                                   it-en     it-es     it-fr     it-it   
splits model_name                                                               
1      bernice                         0.310361  0.478532  0.483648  0.404193   
       bert-base-multilingual-uncased  0.266478  0.275751  0.302653  0.297472   
       mdeberta-v3-base                0.130435  0.130435  0.130435  0.130435   
       twitter-xlm-roberta-base        0.778725  0.782729  0.746078  0.765960   
       xlm-roberta-base                0.152211  0.166127  0.170944  0.164222   

trained_on                               metric  
eval_on                                          
splits model_name                                
1      bernice                         f1_macro  
       bert-base-multilingual-uncased  f1_macro  
       mdeberta-v3-base                f1_macro  
       twitter-xlm-roberta-base        f1_macro  
       xlm-roberta-base                f1_macro  

[5 rows x 27 columns]

In [10]:
print(ROOT.joinpath(f'{exp}_avg_of_5runs.csv'))
# all_eval.to_csv(ROOT.joinpath(f'{exp}_avg_of_5runs.csv'))

/gaueko0/users/nmishra/multiling_fludetection/final_evals/eval_revisedcateg/testset0.6_0.2_0.2/es_learningcurve_predictions_avg_of_5runs.csv


In [14]:
# all_eval.to_csv(ROOT.joinpath(f'all{exp}_avg_of_5runs.csv'))